In [ ]:
# 혼공머신

In [ ]:
# Chapter 05 트리 알고리즘

In [ ]:
# 05-2 교차 검증과 그리드 서치

In [ ]:
# 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞추게 됨
# 테스트 세트를 사용하지 않고 이를 츶겅하는 간단한 방법은 훈련 세트를 또 나누는 것
# 이 데이터를 검증 세트라고 부름

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈련세트(sub_input,sub_target)와 검증세트(val_input, val_target)를 만듦
# test_size 매개변수를 0.2로 지정하여 train_input의 20%를 val_input으로 만듦
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
# 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [4]:
# 모델 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target)) 
print(dt.score(val_input, val_target))
# 과대적합

0.9971133028626413
0.864423076923077


In [6]:
# 검증 세트를 만드느라 훈련 세트가 줄음
# 하지만 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정
# 이럴 때 교차 검증 이용
# 사이킷런에는 cross-validate()라는 교차 검증 함수가 있음

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit_time : 모델을 훈련하는 시간
# score_time : 검증하는 시간
# cross_validate() 함수는 기본저긍로 5-폴드 교차검증을 수행하기에 각 키마다 5개의 숫자가 담겨 있음

{'fit_time': array([0.0124824 , 0.01053929, 0.01416039, 0.01151323, 0.01049829]), 'score_time': array([0.00144362, 0.00126648, 0.00131297, 0.00138521, 0.00196195]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
# 분할기를 사용한 교차 검증
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음
# 훈련 세트를 섞으려면 분할기를 지정해야 함
# 분류 모델의 경우 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold 
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 만약 후련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성
# n_splits는 매개변수 몇 폴드 교차 검증을 할지 정함
splitter =StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [12]:
# max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 바뀜.
# 사이킷런의 GridSearchCV 틀래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
from sklearn.model_selection import GridSearchCV
params = { 'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# GridSearchCV 틀래스 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦

In [13]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
# gs 객체에 fit()메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 mimn_impurity_decrease 값을 바꿔가며 5번 실행
# GridSearchCV의 cv 매개변수 기본값은 5
# min_impurity_decrease 값마다 5-폴드 교차 검증을 수행 
# 결국 5X5=25 개의 모델을 훈련
# 많은 모델을 훈련하기 때문에 GridSearchCV 클래스의 n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋음 . -1로 지정하면 모든 시스템 내 모든 코어 사용
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
# 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
# 이 모델은 gs객체의 best_estimator_ 속성에 저장되어 있음
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_  속성에 저장되어 있음
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score 키에 저장되어 있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
# 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스 추출 가능
# 그 다음 이 인덱스를 이용해 parpams 키에 저장된 매개변수를 출력할 수 있음
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [21]:
# 조금 더 복잡한 매개변수 조합
# min_impurity_decrese : 노드를 분할하기 위한 불순도 감소 최소량을 지정
# min_samples_split : 노드를 나누기 위한 최소 샘플 수
# arange() : 첫번째 매개변수 값에서 두번째 매개변수까지 세번째 매개변수를 계속 더한 배열 만듦
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)
          }

In [23]:
# 그리드 서치 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [24]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [25]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [28]:
# 랜덤 서치
# 매개변수의 값이 수치일 때 갑스이 범위나 간격을 미리 정하기 어려울 수 있음
# 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시가닝 오래 걸릴 수 있음
# 이럴 때 랜덤 서치 사용
# 랜덤 서치에는 매개변수 값이 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

# uniform과 randint 클래스는 주어진 벙위에서 고르게 값을 뽑음. randint는 정숫값. uniform은 실숫값
from scipy.stats import uniform, randint

# 탐색할 매개변수의 딕셔너리를 만듦
# min_samples_leaf : 리프 노드가 되기 위한 최쇠 샘플의 개수
params = { 'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25),}

In [29]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 저장
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc56aee0a10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc5698fe410>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc569b610d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc569b61850>},
                   random_state=42)

In [30]:
# 최적의 매개변수 조합
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [32]:
# 최고의 교차 검증 점수
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [33]:
# 최적의 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장됨
# 이 모델을 최종 모델로 결정하고 테스트 세트의 성능 확인
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86
